In [ ]:
import ipyvuetify as v
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from IPython.display import display, clear_output

### SQLytes Inventory Tracking



In [ ]:
backend = 'http://127.0.0.1:5000/sqlytes/'
# output_area = widgets.Output()

In [ ]:
class WarehouseWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_warehouses = self.getAvailableWarehouses()
        self.warehouses = self.getAllWarehouses().get('Warehouses')
        self.current_warehouse_index = 0
        self.all_warehouses_df = pd.DataFrame()
       
        self.warehouses_options_dropdown = widgets.Select(
            options=['All Warehouses info', 'All parts for a warehouse'],
            value='All Warehouses info',
            description='Search for:',
            rows=2,
        )
        
        #self.warehouses_options_dropdown.observe(self.warehouse_options_dropdown_eventhandler, names='value')

        self.warehouse_selection = widgets.Dropdown(
            options=self.available_warehouses,
            value=self.available_warehouses[0],
            description="Select wid:"
        )
        
        self.warehouse_selection.observe(self.on_selection_change)
        
        self.next_warehouse_button = widgets.Button(
            description="Next Warehouse",
            icon='check'
        )
        
    def getAvailableWarehouses(self):
        """Return all IDs of the warehouses in the Data Base."""
        response = requests.get(f'{backend}{"warehouse"}')
        # List to store all available wid's.
        wids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data['Warehouses']:
                wids.append(record['wid'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return wids
    
    def getAllWarehouses(self):
        """Return all warehouses in the Data Base."""
        response = requests.get(f'{backend}{"warehouse"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data

        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}
    
    def displayNextWarehouse(self,_):
        self.current_warehouse_index = (self.current_warehouse_index+1) % len(self.warehouses)
        print(f"Displaying index {self.current_warehouse_index}")
        selected_warehouse = self.warehouses[self.current_warehouse_index]
        print(f"Displaying Warehouse ID: {selected_warehouse}")
        selected_warehouse_df = pd.DataFrame.from_dict(selected_warehouse, orient="index")
        self.all_warehouses_df = selected_warehouse_df
        with self.output_area:
            clear_output(wait=True)
            display(self.all_warehouses_df)
            
    def getWarehousePartsAPI(self,selected_warehouse):
        """Obtain all Parts for a warehouse.

        Args:
            wid (int): ID of the warehouse of which the parts are going to be showed.
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'warehouse/{selected_warehouse}/parts'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                all_parts_df = pd.DataFrame(data["WarehouseParts"])
                display(all_parts_df)
            elif response.status_code == 404:
                print('Error {}: Warehouse has no parts'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_warehouse = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_warehouse = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getWarehousePartsAPI(selected_warehouse)
    

### Select Warehouse to search the parts for

In [ ]:
warehouseWidgets = WarehouseWidgets()

In [ ]:
# warehouseWidgets.warehouses_options_dropdown.observe(warehouseWidgets.warehouse_options_handler,names='value')

In [ ]:
# Display the dropdown initially
#display(warehouseWidgets.warehouses_options_dropdown)

In [ ]:
warehouseWidgets.getWarehousePartsAPI(warehouseWidgets.warehouse_selection.value)
warehouseWidgets.warehouse_selection.observe(warehouseWidgets.on_selection_change)
display(warehouseWidgets.warehouse_selection)
display(warehouseWidgets.output_area)

In [ ]:
# display(warehouseWidgets.warehouses_options_dropdown)

In [ ]:
# warehouseWidgets.displayNextWarehouse(_)

In [ ]:
# # Attach the function to the button's click event
# button = warehouseWidgets.next_warehouse_button
# button.on_click(warehouseWidgets.displayNextWarehouse)

# # Display the widgets
# display(warehouseWidgets.output_area)
# display(button)